In [1]:
import os
os.environ['SNORKELDB'] = 'sqlite:///brat-import.db'

from snorkel import SnorkelSession
from snorkel.contrib.brat import *
from snorkel.parsers import StanfordCoreNLPServer
from snorkel.models import Candidate, Document, candidate_subclass, GoldLabel

session = SnorkelSession()

In [2]:
corenlp_server = StanfordCoreNLPServer(version='3.6.0', split_newline=False, num_threads=1)

In [3]:
brat = Brat(session)

In [4]:
from snorkel.parsers import TextDocPreprocessor, CorpusParser
doc_preprocessor = TextDocPreprocessor("small_data/brat/*.txt", encoding="utf-8")
corpus_parser = CorpusParser()
corpus_parser.apply(doc_preprocessor, parallelism=1)

Clearing existing...
Running UDF...


In [5]:
input_dir = "small_data/brat/"
brat.import_project(input_dir, annotations_only=False, annotator_name='brat', num_threads=1, parser=corenlp_server)

NotImplementedError: Discontinuous Spans
NotImplementedError: Discontinuous Spans
NotImplementedError: Discontinuous Spans


Causation T1 T2
R1-1 Causation T1 T2-0
R1-2 Causation T1 T2-1
Causation T3 T4
R2-1 Causation T3 T4-0
R2-2 Causation T3 T4-1
Negative T5 T7
R3 Negative T5 T7
Causation T5 T6
R4 Causation T5 T6
Causation T8 T9
R5 Causation T8 T9
Causation T10 T12
R6-1 Causation T10 T12-0
R6-2 Causation T10 T12-1
Causation T10 T11
R7 Causation T10 T11
Causation T13 T14
R8 Causation T13 T14
Causation T15 T16
R9 Causation T15 T16
Causation T17 T18
R10 Causation T17 T18
Causation T19 T20
R11 Causation T19 T20
Causation T21 T22
R12 Causation T21 T22
Negative T23 T24
R13 Negative T23 T24
Causation T27 T25
R14 Causation T27 T25
Causation T26 T25
R15 Causation T26 T25
Causation T28 T29
R16 Causation T28 T29
Causation T30 T31
R17 Causation T30 T31
Causation T1 T2
R1 Causation T1 T2
Negative T4 T5
R2 Negative T4 T5
Negative T3 T5
R3 Negative T3 T5
Causation T11 T12
R4 Causation T11 T12
Causation T18 T19
R5 Causation T18 T19
Causation T6 T8
R6 Causation T6 T8
Causation T7 T8
R7 Causation T7 T8
Causation T27 T29
R8 

In [6]:
# each entity/relation type is assigned to a different split
#for i in range(0,1):
candidates = session.query(Candidate).filter(Candidate.split == 0).all()
print len(candidates)
for i,c in enumerate(candidates):
    print type(c).type, c
    if i > 5:
        break
print
    
# ProteinComponent relations
candidates = session.query(Candidate).filter(Candidate.split == 0).all()

52
Causation.type Causation(Span("sinal7-2", sentence=900, chars=[16,23], words=[3,3]), Span("produce flower buds of abnormal shape, characterized by presence of cavities in the bud tips", sentence=900, chars=[38,129], words=[7,22]))
Causation.type Causation(Span("rha1a-1", sentence=623, chars=[9,15], words=[2,2]), Span("higher number of leaves", sentence=623, chars=[140,162], words=[25,28]))
Causation.type Causation(Span("csu1-4", sentence=550, chars=[4,9], words=[1,1]), Span("rosette was clearly smaller", sentence=550, chars=[18,44], words=[3,6]))
Causation.type Causation(Span("csu1-4", sentence=550, chars=[4,9], words=[1,1]), Span("showed a yellowish coloration", sentence=550, chars=[61,89], words=[10,13]))
Causation.type Causation(Span("sinal7-2", sentence=727, chars=[66,73], words=[12,12]), Span("higher SOL values", sentence=727, chars=[82,98], words=[14,16]))
Causation.type Causation(Span("rha1a-1", sentence=727, chars=[54,60], words=[10,10]), Span("higher SOL values", sentence=7

In [7]:
from snorkel.viewer import SentenceNgramViewer

sv = SentenceNgramViewer(candidates, session=session, n_per_page=6, height=400,
                         annotator_name='brat')
sv

<IPython.core.display.Javascript object>